# Questionnaire

Nodes in the questionnaire have two possible types: **Questions** or **Actions**:

* **Questions**: interact with the user;
* **Actions**: process user input. Actions are stored in the user **profile**.

In [1]:
class Node:
    def __init__(self, id, text):
        self.id = id
        self.text = text
        self.num_children = 0
        
    def __repr__(self):
        return f"<{self.type} {self.id}: {self.text}>"
        
    def setNext(self, l):
        self.num_children=len(l)
        self.next = l
        
    def getNext(self, i):
        try:
            return self.next[i]
        except:
            return None
    
class Action(Node):
    def __init__(self, id, text):
        super().__init__(id, text)
        self.type='action'
        
class Question(Node):
    def __init__(self, id, text):
        super().__init__(id, text)
        self.type='question'

Questionnaires store and run the questions/actions graph. They are built from a json-like object (dict, in Python).

In [2]:
class Questionnaire:
    def __init__(self, q):
        # Set actions and questions
        # The first question must be named q0
        actions = q.get('actions')
        questions = q.get('questions')
            
        self.nodes = dict()
        self.actions = dict()
        for a_id, a in actions.items():
            self.actions[a_id] = Action(a_id,a.get("text"))
            self.nodes[a_id] = self.actions[a_id]
        
        self.questions = dict()
        for q_id, q in questions.items():
            self.questions[q_id] = Question(q_id, q.get("text"))
            self.nodes[q_id] = self.questions[q_id]
            
        # set children
        for q_id, q in self.questions.items():
            q.setNext( [ self.nodes.get(i) for i in questions.get(q_id).get("children")] )
        for a_id, a in self.actions.items():
            a.setNext( [ self.nodes.get(i) for i in actions.get(a_id).get("children")] )
            
            
                      
    def run(self):
        currNode = self.questions.get('q0')
        results = []

        while True:
            if currNode.type=='question':
                inp = int (input(f"{currNode.text}"))
                currNode = currNode.next[inp]

            elif currNode.type=='action':
                
                # action is a leaf
                if currNode.num_children==0:
                    results += [ currNode.id ]
                    return results
                
                # action is not a leaf
                elif currNode.num_children==1:
                    results = results + [ currNode.id ]
                    currNode = currNode.next[0]

                # Multiple actions possible: list all and record the selected one
                else:
                    results += [ currNode.id ]
                    for i,a in enumerate(currNode.next):
                        print(f"Opção {i}:",a.text)
                        
                    inp = int(input("Escolha uma opção acima"))
                    
                    currNode = currNode.getNext(inp)

        return results

In [3]:
questionnaire = {
    'questions':{
        'q0': { "text": "O solo está degradado?", "children": ['q1','a0'] },
        'q1': { "text": "Tem pecuária na vizinhança?", "children": ['q2','a1'] },
        'q2': { "text": "Tem risco de incêndio?", "children":['a3','a2'] },
        'q3': { "text": "Qual o potencial de regeneração natural? (0=baixo, 1=alto)", "children":['a5', 'a6'] },
    },
    'actions':{
        'a0': { "text": "Recuper o solo", "children": ['q1'] },
        'a1': { "text": "Cercar", "children": ['q2'] },
        'a2': { "text": "Acerar", "children": ['a3'] },
        'a3': { "text": "Restauração", "children": ['q3'] },
        'a4': { "text": "Restauração com produtividade saf/floresta de uso múltiplo", "children": []},
        'a5': { "text": "Baixo potencial de regeneração natural", "children": ['a9','a10'] },
        'a6': { "text": "Alto potencial de regeneração natural", "children": ['a7','a8'] },
        'a7': { "text": "Permitir regeneração natural", "children":[] },
        'a8': { "text": "Manejar a regeneração natural", "children":[] },
        'a9': { "text": "Plantio de mudas", "children": [] },
        'a10': { "text": "Semeadura direta", "children": ['a11','a12'] },
        'a11': { "text": "Semeadura direta em área total", "children": [] },
        'a12': { "text": "Semeadura direta em linha", "children": [] }
    }
}

In [4]:
quest = Questionnaire(questionnaire)

In [5]:
res = quest.run()

O solo está degradado?1
Tem pecuária na vizinhança?0
Tem risco de incêndio?0
Qual o potencial de regeneração natural? (0=baixo, 1=alto)0
Opção 0: Plantio de mudas
Opção 1: Semeadura direta
Escolha uma opção acima1
Opção 0: Semeadura direta em área total
Opção 1: Semeadura direta em linha
Escolha uma opção acima0


In [6]:
res

['a0', 'a3', 'a5', 'a10', 'a11']